# Describe the business problem

# This notebook describes how to use CPLEX Modeling for Python meet demand with your resources.

# Step 1: create a model instance

In [33]:
from docplex.mp.model import Model
mdl = Model(name='SimplestResourceAllocation')

# Step 2: Create decision variables

### We have 4 products to manufacture

Decision variables:

$x1$ = number of product1

$x2$ = number of product2

$x3$ = number of product3

$x4$ = number of product4

In [34]:
x1 = mdl.continuous_var(name='x1')
x2 = mdl.continuous_var(name='x2')
x3 = mdl.continuous_var(name='x3')
x4 = mdl.continuous_var(name='x4')

# Step 3: Add an objective
Objective here is to maximize profit. 

Profit per product:

product1: \$20.00

product2: \$12.00

product3: \$40.00

product4: \$25.00

Profit expression:$$
profit = 20 * x1 + 12 * x2 + 40 * x3 + 25 *x4
$$Objective:$$
maximize(profit)
$$

In [35]:
mdl.maximize(20 * x1 + 12 * x2 + 40 * x3 + 25 * x4)

In [36]:
mdl.solve()
mdl.print_information()

Model: SimplestResourceAllocation
 - number of variables: 4
   - binary=0, integer=0, continuous=4
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - problem type is: LP


# Step 4: Add constraints
We want to make ensure total manpower required for all these products equals to 50. So we add constraint 

Constraint:
x1 + x2 + x3 + x4 <= 50


In [37]:
mdl.add_constraint(x1 + x2 + x3 + x4 <= 50)

docplex.mp.LinearConstraint[](x1+x2+x3+x4,LE,50)

##### This leads to picking only product3, because these have the highest profit

In [38]:
mdl.solve()
mdl.print_solution()

objective: 2000.000
  x3=50.000


In [39]:
mdl.add_constraint(3*x1+2*x2+x3 <= 100) #raw material A

docplex.mp.LinearConstraint[](3x1+2x2+x3,LE,100)

In [40]:
mdl.add_constraint(x2+ 2*x3 +3* x4 <= 90) #raw material B

docplex.mp.LinearConstraint[](x2+2x3+3x4,LE,90)

# Step 5: solve the model

In [41]:
msol = mdl.solve()
if msol is not None:
    mdl.report()
    mdl.print_solution()

* model SimplestResourceAllocation solved with objective = 1900.000
objective: 1900.000
  x1=5.000
  x3=45.000


# Step 6: extract the solution
Extract the optimized value of the decision variables.

In [42]:
x1_sol = x1.solution_value
x2_sol = x2.solution_value
x3_sol = x3.solution_value
x4_sol = x4.solution_value
print(f"x1={x1_sol}, x2={x2_sol}, x3={x3_sol}, x4={x4_sol}")

x1=5.0, x2=0, x3=45.0, x4=0


# Debugging: show the model
Show the model in a text respresentation (.lp file format).
Note that the constraints do not look exactly as created. CPLEX has re-arranged the variables in the expressions.

In [43]:
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: SimplestResourceAllocation

Maximize
 obj: 20 x1 + 12 x2 + 40 x3 + 25 x4
Subject To
 c1: x1 + x2 + x3 + x4 <= 50
 c2: 3 x1 + 2 x2 + x3 <= 100
 c3: x2 + 2 x3 + 3 x4 <= 90

Bounds
End



# Conclusion
Above shows the basic steps to build and solve a CPLEX optimization model.
However, you would never build a real-life model like this. For one, we have hard-coded the demand, the decision variables and constraints.

In practice, we want a model that adapts to the current state. So we have to consider how the model needs to be able to adapt and how that information can be supplied in the form of input data tables.